<a href="https://colab.research.google.com/github/Zuhair0000/FYP-Research/blob/main/FYP_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# List all files in the Colab working directory
print(os.listdir("/content"))

['.config', 'Dataset B', 'Dataset A', 'sample_data', '.ipynb_checkpoints']


In [ ]:
import glob
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

all_files = glob.glob("/content/Dataset A/*.xlsx")
print("All Excel files:", all_files)

# Separate solar and wind
solar_files = [f for f in all_files if "光伏" in f]
wind_files  = [f for f in all_files if "风力" in f]

print("Solar files:", solar_files)
print("Wind files:", wind_files)

All Excel files: ['/content/Dataset A/风力春季发电.xlsx', '/content/Dataset A/风力冬季发电.xlsx', '/content/Dataset A/光伏冬季发电.xlsx', '/content/Dataset A/风力秋季发电.xlsx', '/content/Dataset A/风力夏季发电.xlsx', '/content/Dataset A/光伏夏季发电.xlsx', '/content/Dataset A/光伏秋季发电.xlsx', '/content/Dataset A/光伏春季发电.xlsx']
Solar files: ['/content/Dataset A/光伏冬季发电.xlsx', '/content/Dataset A/光伏夏季发电.xlsx', '/content/Dataset A/光伏秋季发电.xlsx', '/content/Dataset A/光伏春季发电.xlsx']
Wind files: ['/content/Dataset A/风力春季发电.xlsx', '/content/Dataset A/风力冬季发电.xlsx', '/content/Dataset A/风力秋季发电.xlsx', '/content/Dataset A/风力夏季发电.xlsx']


In [ ]:

# Load solar
solar_dfs = [pd.read_excel(f) for f in solar_files]
solar_data = pd.concat(solar_dfs, ignore_index=True)

# Load wind
wind_dfs = [pd.read_excel(f) for f in wind_files]
wind_data = pd.concat(wind_dfs, ignore_index=True)

print("Solar shape:", solar_data.shape)
print("Wind shape:", wind_data.shape)

Solar shape: (8753, 7)
Wind shape: (8752, 5)


In [ ]:
print("Solar columns:", solar_data.columns)
print(solar_data.head())

print("\nWind columns:", wind_data.columns)
print(wind_data.head())

print("\nMissing values in Solar:\n", solar_data.isnull().sum())
print("\nMissing values in Wind:\n", wind_data.isnull().sum())

Solar columns: Index(['年份', '时间', '温度', '湿度', '地面辐射强度', '大气上层辐射强度', '光伏发电'], dtype='object')
                   年份  时间      温度     湿度  地面辐射强度  大气上层辐射强度  光伏发电
0 2019-12-01 00:00:00   1 -10.687  0.001     0.0       0.0   0.0
1 2019-12-01 01:00:00   2 -10.623  0.001     0.0       0.0   0.0
2 2019-12-01 02:00:00   3 -10.564  0.001     0.0       0.0   0.0
3 2019-12-01 03:00:00   4 -10.954  0.001     0.0       0.0   0.0
4 2019-12-01 04:00:00   5 -11.344  0.001     0.0       0.0   0.0

Wind columns: Index(['年份', '时间', '空气密度', '风速', '发电功率'], dtype='object')
                   年份  时间   空气密度     风速     发电功率
0 2019-03-01 00:00:00   1  1.282  0.955    0.020
1 2019-03-01 01:00:00   2  1.270  0.881    0.010
2 2019-03-01 02:00:00   3  1.260  1.627    1.825
3 2019-03-01 03:00:00   4  1.253  2.421   26.870
4 2019-03-01 04:00:00   5  1.249  3.450  165.285

Missing values in Solar:
 年份          0
时间          0
温度          0
湿度          0
地面辐射强度      0
大气上层辐射强度    0
光伏发电        0
dtype: int64

Missing val

In [ ]:
# --- Solar ---
solar_data = solar_data.rename(columns={
    "年份": "DateTime",
    "时间": "Index",
    "温度": "Temperature",
    "湿度": "Humidity",
    "地面辐射强度": "Ground_Irradiance",
    "大气上层辐射强度": "Atmospheric_Irradiance",
    "光伏发电": "Solar_Power"
})

# --- Wind ---
wind_data = wind_data.rename(columns={
    "年份": "DateTime",
    "时间": "Index",
    "空气密度": "Air_Density",
    "风速": "Wind_Speed",
    "发电功率": "Wind_Power"
})

In [ ]:
solar_data["DateTime"] = pd.to_datetime(solar_data["DateTime"])
wind_data["DateTime"]  = pd.to_datetime(wind_data["DateTime"])

# Sort by datetime
solar_data = solar_data.sort_values("DateTime")
wind_data  = wind_data.sort_values("DateTime")
solar_data


,DateTime,Index,Temperature,Humidity,Ground_Irradiance,Atmospheric_Irradiance,Solar_Power
744,2019-01-01 08:00:00,745,-19.889,0.001,82.680,209.252,257.690
745,2019-01-01 09:00:00,746,-19.004,0.001,149.553,345.604,530.391
746,2019-01-01 10:00:00,747,-17.971,0.001,193.625,430.654,871.016
747,2019-01-01 11:00:00,748,-17.281,0.001,203.808,458.550,1087.304
748,2019-01-01 12:00:00,749,-16.738,0.001,185.193,427.391,1010.161
...,...,...,...,...,...,...,...
739,2019-12-31 19:00:00,740,-23.621,0.000,0.000,0.000,0.000
740,2019-12-31 20:00:00,741,-23.980,0.000,0.000,0.000,0.000
741,2019-12-31 21:00:00,742,-24.826,0.000,0.000,0.000,0.000
742,2019-12-31 22:00:00,743,-26.172,0.000,0.000,0.000,0.000


In [ ]:
solar_data = solar_data.drop(columns=["Index"])
wind_data  = wind_data.drop(columns=["Index"])

In [ ]:
print("Solar date range:", solar_data["DateTime"].min(), "→", solar_data["DateTime"].max())
print("Wind date range:", wind_data["DateTime"].min(), "→", wind_data["DateTime"].max())

print("\nSolar frequency check:")
print(solar_data["DateTime"].diff().value_counts().head())

print("\nWind frequency check:")
print(wind_data["DateTime"].diff().value_counts().head())

Solar date range: 2019-01-01 08:00:00 → 2019-12-31 23:00:00
Wind date range: 2019-01-01 08:00:00 → 2019-12-31 23:00:00

Solar frequency check:
DateTime
0 days 01:00:00    8751
0 days 00:00:00       1
Name: count, dtype: int64

Wind frequency check:
DateTime
0 days 01:00:00    8751
Name: count, dtype: int64


In [ ]:
# Resample Solar
solar_data = solar_data.set_index("DateTime").resample("h").mean().interpolate()

# Resample Wind
wind_data = wind_data.set_index("DateTime").resample("h").mean().interpolate()

# Reset index back
solar_data = solar_data.reset_index()
wind_data  = wind_data.reset_index()

In [ ]:
merged_data = pd.merge_asof(
    solar_data.sort_values("DateTime"),
    wind_data.sort_values("DateTime"),
    on="DateTime",
    direction="nearest"  # matches closest timestamp
)
merged_data

,DateTime,Temperature,Humidity,Ground_Irradiance,Atmospheric_Irradiance,Solar_Power,Air_Density,Wind_Speed,Wind_Power
0,2019-01-01 08:00:00,-19.889,0.001,82.680,209.252,257.690,1.335,6.178,1203.800
1,2019-01-01 09:00:00,-19.004,0.001,149.553,345.604,530.391,1.337,6.534,1410.415
2,2019-01-01 10:00:00,-17.971,0.001,193.625,430.654,871.016,1.339,6.747,1547.633
3,2019-01-01 11:00:00,-17.281,0.001,203.808,458.550,1087.304,1.340,6.765,1556.072
4,2019-01-01 12:00:00,-16.738,0.001,185.193,427.391,1010.161,1.342,6.606,1456.741
...,...,...,...,...,...,...,...,...,...
8747,2019-12-31 19:00:00,-23.621,0.000,0.000,0.000,0.000,1.385,6.260,1254.654
8748,2019-12-31 20:00:00,-23.980,0.000,0.000,0.000,0.000,1.387,5.784,995.959
8749,2019-12-31 21:00:00,-24.826,0.000,0.000,0.000,0.000,1.388,4.955,614.468
8750,2019-12-31 22:00:00,-26.172,0.000,0.000,0.000,0.000,1.388,4.525,457.358


In [ ]:
for lag in [1, 3, 6, 24]:  # 1h, 3h, 6h, 24h lags
    merged_data[f"Solar_Power_lag{lag}"] = merged_data["Solar_Power"].shift(lag)
    merged_data[f"Wind_Power_lag{lag}"] = merged_data["Wind_Power"].shift(lag)

In [ ]:
merged_data["Solar_Power_roll6h"] = merged_data["Solar_Power"].rolling(window=6).mean()
merged_data["Wind_Power_roll6h"] = merged_data["Wind_Power"].rolling(window=6).mean()

In [ ]:
merged_data["hour"] = merged_data["DateTime"].dt.hour
merged_data["dayofweek"] = merged_data["DateTime"].dt.dayofweek
merged_data["month"] = merged_data["DateTime"].dt.month

In [ ]:
merged_data = merged_data.dropna().reset_index(drop=True)

In [ ]:
# Path to the clean India dataset
file_path = "/content/Dataset B/January 2024- June 2025.xlsx"

# Load the Excel file
india_df = pd.read_excel(file_path)

# Inspect first rows
print(india_df.head())
print(india_df.shape)

             Timestamp Demand (MW) Wind (MW) Solar (MW) Total Generation (MW)
0  01-01-2024 00:00:00   141981.26   9545.07      20.29             142193.38
1  01-01-2024 01:00:00   138675.49   9027.05      19.96             138792.04
2  01-01-2024 02:00:00   136122.78   8241.48      19.79             136796.02
3  01-01-2024 03:00:00   135806.51   7866.09      19.73             136859.67
4  01-01-2024 04:00:00   139905.77   7430.71      20.02             140561.49
(12990, 5)


In [ ]:
# Convert Timestamp column to datetime
india_df["Timestamp"] = pd.to_datetime(india_df["Timestamp"], errors="coerce")

# Drop rows where Timestamp is missing
india_df = india_df.dropna(subset=["Timestamp"])

# Convert numeric columns (they may be read as strings)
num_cols = ["Demand (MW)", "Wind (MW)", "Solar (MW)", "Total Generation (MW)"]
for col in num_cols:
    india_df[col] = pd.to_numeric(india_df[col], errors="coerce")

# Sort chronologically
india_df = india_df.sort_values("Timestamp").reset_index(drop=True)

print(india_df.info())
print(india_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5184 entries, 0 to 5183
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Timestamp              5184 non-null   datetime64[ns]
 1   Demand (MW)            5184 non-null   float64       
 2   Wind (MW)              5184 non-null   float64       
 3   Solar (MW)             5184 non-null   float64       
 4   Total Generation (MW)  5184 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 202.6 KB
None
                 Timestamp    Demand (MW)     Wind (MW)    Solar (MW)  \
count                 5184    5184.000000   5184.000000   5184.000000   
mean   2024-10-21 00:50:00  192456.737118   8740.483189  15899.132461   
min    2024-01-01 00:00:00   97065.280000    636.000000    -49.270000   
25%    2024-05-06 23:45:00  182160.462500   4536.230000     96.275000   
50%    2024-09-21 23:30:00  195634.545000   7645.9

In [ ]:
# Select subset of columns
india_df = india_df[["Timestamp", "Demand (MW)", "Wind (MW)", "Solar (MW)", "Total Generation (MW)"]]

print(india_df.head())

            Timestamp  Demand (MW)  Wind (MW)  Solar (MW)  \
0 2024-01-01 00:00:00    141981.26    9545.07       20.29   
1 2024-01-01 01:00:00    138675.49    9027.05       19.96   
2 2024-01-01 02:00:00    136122.78    8241.48       19.79   
3 2024-01-01 03:00:00    135806.51    7866.09       19.73   
4 2024-01-01 04:00:00    139905.77    7430.71       20.02   

   Total Generation (MW)  
0              142193.38  
1              138792.04  
2              136796.02  
3              136859.67  
4              140561.49  
